## converting data from xarray to pandas format

In [1]:
import numpy as np
import pandas as pn
import xarray as xr
import matplotlib.pyplot as plt


### Saildrone data in xarray form

In [2]:
url = 'https://podaac-opendap.jpl.nasa.gov/opendap/hyrax/allData/insitu/L2/saildrone/Baja/saildrone-gen_4-baja_2018-sd1002-20180411T180000-20180611T055959-1_minutes-v1.nc'
ds = xr.open_dataset(url)
ds

<xarray.Dataset>
Dimensions:               (obs: 86839, trajectory: 1)
Coordinates:
  * trajectory            (trajectory) float32 1002.0
    time                  (trajectory, obs) datetime64[ns] ...
    latitude              (trajectory, obs) float64 ...
    longitude             (trajectory, obs) float64 ...
Dimensions without coordinates: obs
Data variables:
    SOG                   (trajectory, obs) float64 ...
    COG                   (trajectory, obs) float64 ...
    HDG                   (trajectory, obs) float64 ...
    HDG_WING              (trajectory, obs) float64 ...
    ROLL                  (trajectory, obs) float64 ...
    PITCH                 (trajectory, obs) float64 ...
    WING_ANGLE            (trajectory, obs) float64 ...
    BARO_PRES_MEAN        (trajectory, obs) float64 ...
    BARO_PRES_STDDEV      (trajectory, obs) float64 ...
    TEMP_AIR_MEAN         (trajectory, obs) float64 ...
    TEMP_AIR_STDDEV       (trajectory, obs) float64 ...
    RH_MEAN        

In [3]:
# swap obs for time
ds=ds.isel(trajectory=0)
ds = ds.swap_dims({'obs':'time'})
ds

<xarray.Dataset>
Dimensions:               (time: 86839)
Coordinates:
    trajectory            float32 1002.0
  * time                  (time) datetime64[ns] 2018-04-11T18:00:00 ... 2018-06-11T05:57:00
    latitude              (time) float64 ...
    longitude             (time) float64 ...
Data variables:
    SOG                   (time) float64 ...
    COG                   (time) float64 ...
    HDG                   (time) float64 ...
    HDG_WING              (time) float64 ...
    ROLL                  (time) float64 ...
    PITCH                 (time) float64 ...
    WING_ANGLE            (time) float64 ...
    BARO_PRES_MEAN        (time) float64 ...
    BARO_PRES_STDDEV      (time) float64 ...
    TEMP_AIR_MEAN         (time) float64 ...
    TEMP_AIR_STDDEV       (time) float64 ...
    RH_MEAN               (time) float64 ...
    RH_STDDEV             (time) float64 ...
    TEMP_IR_UNCOR_MEAN    (time) float64 ...
    TEMP_IR_UNCOR_STDDEV  (time) float64 ...
    UWND_MEAN   

In [ ]:
#resample to make time = 1 hr increments
dshr=ds.SOG.resample(time='1h', skipna=True, label='left').mean()
dshr
# does data need to be averaged from beginning of hour to end, or from half way through one hour and half through the other??
#for now, we will average hours from beginning of hour to the end, how resample defaults

# do the math stuff in xarray
#then save as a dataframe, then to strings of the data 

In [15]:
dshr.time = (dshr.time)*.01
dshr.time

TypeError: ufunc multiply cannot use operands with types dtype('<M8[ns]') and dtype('float64')

### Decode and store data as Pandas

In [ ]:
pd_ds = ds.to_dataframe()
pd_ds

In [ ]:
pd_ds[['time', 'PITCH']]

In [ ]:
#write dataframe to file
pd_ds.to_csv("test1.csv")  
#read it back
pd.read_csv("test1.csv").head()

### Convert new csv file to a text file

In [ ]:
# A simple program to create a formatted text file from a *.csv file.

csv_file = input('test1.csv')
txt_file = input('test1.txt')

try:
    my_input_file = open(csv_file, "r")
except IOError as e:
    print("I/O error({0}): {1}".format(e.errno, e.strerror))

if not my_input_file.closed:
    text_list = [];
    for line in my_input_file.readlines():
        line = line.split(",", 2)
        text_list.append(" ".join(line))
    my_input_file.close()

try:
    my_output_file = open(txt_file, "w")
except IOError as e:
    print("I/O error({0}): {1}".format(e.errno, e.strerror))

if not my_output_file.closed:
    my_output_file.write("#1\n")
    my_output_file.write("double({},{})\n".format(len(text_list), 2))
    for line in text_list:
        my_output_file.write("  " + line)
    print('File Successfully written.')
    my_output_file.close()